<a href="https://colab.research.google.com/github/marcelo7bastos/mba_enap_introducao_ciencia_dados/blob/main/trabalhos_realizados/Aula3_Exercicio3_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercício 3.1

In [ ]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')


Authenticated


In [ ]:
import pandas as pd
## Defina o id do seu projeto no bigquery!!!!!
project_id = '365846072239' # Defina o id do seu projeto no bigquery!!!!!
## Defina o id do seu projeto no bigquery!!!!!

df = pd.io.gbq.read_gbq('''
                        SELECT
                          d.id_municipio,
                          d.nome AS nome_municipio,
                          d.sigla_uf,
                          pop.ano,
                          PARSE_DATE('%Y', CAST(pop.ano AS STRING)) AS data_ano,
                          pop.populacao,
                          pib.pib,
                          ROUND(SAFE_DIVIDE(pib.pib, pop.populacao), 2) AS pib_per_capita
                        FROM `basedosdados.br_bd_diretorios_brasil.municipio` AS d

                        -- junta com população (se não tiver população, fica NULL)
                        LEFT JOIN `basedosdados.br_ibge_populacao.municipio` AS pop
                          ON d.id_municipio = pop.id_municipio
                          AND pop.ano BETWEEN 2002 AND 2018

                        -- junta com pib (se não tiver pib, fica NULL)
                        LEFT JOIN `basedosdados.br_ibge_pib.municipio` AS pib
                          ON d.id_municipio = pib.id_municipio
                        AND pop.ano = pib.ano
                        AND pop.ano BETWEEN 2002 AND 2018
                        --LIMIT 100
                        ;''',
                        project_id=project_id)

df.head()

/tmp/ipython-input-4134288332.py:6: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  df = pd.io.gbq.read_gbq('''


,id_municipio,nome_municipio,sigla_uf,ano,data_ano,populacao,pib,pib_per_capita
0,1100809,Candeias do Jamari,RO,2002,2002-01-01,14314,61407000,4290.00
1,1100809,Candeias do Jamari,RO,2003,2003-01-01,14874,71276000,4791.99
2,1100809,Candeias do Jamari,RO,2004,2004-01-01,17338,93368000,5385.17
3,1100809,Candeias do Jamari,RO,2005,2005-01-01,16700,102722000,6151.02
4,1100809,Candeias do Jamari,RO,2006,2006-01-01,17346,110797000,6387.47


In [2]:
## Explique o que fez na sua query e por que.
"""
Explicação da Query

1. Base de municípios (diretório):
   - A query começa pela tabela `br_bd_diretorios_brasil.municipio`.
   - Essa tabela é usada como referência porque contém todos os municípios do Brasil
     com código IBGE, nome e sigla da UF.
   - Garantimos que todos os municípios apareçam, mesmo que faltem dados de PIB ou população.

2. Join com população:
   - Foi feito um LEFT JOIN com `br_ibge_populacao.municipio`.
   - Isso significa que, se não existir dado de população para algum município/ano,
     o município ainda aparece, mas a coluna de população fica NULL.
   - O filtro `pop.ano BETWEEN 2002 AND 2018` foi aplicado para restringir os anos ao período desejado.

3. Join com PIB:
   - Foi feito um LEFT JOIN com `br_ibge_pib.municipio`.
   - O join é pelo município e pelo ano, garantindo que PIB e população correspondam ao mesmo período.
   - Também foi colocado o filtro de anos (2002 a 2018) para limitar o intervalo.

4. Seleção de colunas:
   - `id_municipio`, `nome_municipio` e `sigla_uf` identificam cada município.
   - `pop.ano` é o ano da observação.
   - `PARSE_DATE` transforma o ano em um campo de data (formato compatível com Looker Studio).
   - `pop.populacao` e `pib.pib` trazem os valores originais.
   - `ROUND(SAFE_DIVIDE(pib.pib, pop.populacao), 2)` calcula o PIB per capita com segurança
     (evita erro de divisão por zero) e já arredonda para 2 casas decimais.

Observação:
- Comecei fazendo o inner join do PIB com a População e percebi que não era a
  melhor maneiro porque poderia perder dados
- Depois da alteração para inicial pela base de municípios,
  foi incluído o filtro para considerar apenas os anos entre 2002 e 2018.
- Isso garante que a tabela final traga somente os dados do período solicitado,
  deixando de fora anos anteriores ou posteriores.
""";

## Submeta o seu dataframe criando uma tabela no BigQuery

In [ ]:
df.to_gbq("enapdatasets.pibpercapita",
              project_id=project_id,
              chunksize=40000,
              if_exists='replace',
              )

/tmp/ipython-input-3064127241.py:1: FutureWarning: to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
  df.to_gbq("enapdatasets.pibpercapita",
100%|██████████| 1/1 [00:00<00:00, 2345.81it/s]


#### O comando acima cria um dataset chamado enapdatasets e dentro dele uma tabela chamada pibpercapita.
#### Por padrão, os dados são guardados fisicamente nos EUA, onde também estão guardados os dados do projeto basedosdados.
#### Não é possível fazer join de dados guardados em locais distintos

##### Aqui há uma demonstração de como [criar um dataset manualmente](https://youtu.be/h_HRXdket7U)